# Extra Aufgaben

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
PATH = "gdrive/MyDrive/application-project-abgabe"

## Filterung der Datenbank um irrelevante Ergebnisse zu verhindern

In [66]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity, linear_kernel, polynomial_kernel, sigmoid_kernel, rbf_kernel, laplacian_kernel, chi2_kernel, euclidean_distances, manhattan_distances, cosine_distances
import plotly.express as px
import plotly.graph_objects as go
from tqdm import tqdm
import time
from ast import literal_eval

In [4]:
#by_isrc = pd.read_csv('data/by_isrc.csv')
by_isrc = pd.read_csv(f"{PATH}/by_isrc.csv")

In [5]:
by_isrc["country_code"] = by_isrc["isrc"].apply(lambda x: x[:2])

In [6]:
by_isrc["country_code"].unique()

array(['AE', 'AR', 'AT', 'AU', 'AZ', 'BA', 'BC', 'BE', 'BG', 'BR', 'BX',
       'Br', 'CA', 'CH', 'CI', 'CL', 'CN', 'CO', 'CS', 'CZ', 'DE', 'DG',
       'DK', 'EE', 'EG', 'ES', 'FI', 'FR', 'FX', 'GB', 'GN', 'GR', 'GT',
       'GX', 'HK', 'HR', 'HU', 'ID', 'IE', 'IL', 'IN', 'IR', 'IS', 'IT',
       'JM', 'JP', 'KE', 'KR', 'KS', 'LB', 'LK', 'LT', 'LU', 'LV', 'MD',
       'MM', 'MT', 'MU', 'MX', 'MY', 'NG', 'NI', 'NL', 'NO', 'NZ', 'No',
       'PE', 'PH', 'PL', 'PT', 'QM', 'QZ', 'RE', 'RO', 'RS', 'RU', 'SE',
       'SG', 'SI', 'SK', 'SM', 'SW', 'TC', 'TH', 'TR', 'TW', 'UA', 'UK',
       'UR', 'US', 'UY', 'VE', 'VN', 'ZA', 'ZZ', 'ae', 'ca', 'gb', 'il',
       'lv', 'nl', 'qz', 'us', 've'], dtype=object)

Einige Ländercodes sind kleingeschrieben. Diese sollen in Großbuchstaben umgewandelt werden.

In [7]:
by_isrc["country_code"] = by_isrc["country_code"].str.upper()

In [8]:
by_isrc["country_code"].unique()

array(['AE', 'AR', 'AT', 'AU', 'AZ', 'BA', 'BC', 'BE', 'BG', 'BR', 'BX',
       'CA', 'CH', 'CI', 'CL', 'CN', 'CO', 'CS', 'CZ', 'DE', 'DG', 'DK',
       'EE', 'EG', 'ES', 'FI', 'FR', 'FX', 'GB', 'GN', 'GR', 'GT', 'GX',
       'HK', 'HR', 'HU', 'ID', 'IE', 'IL', 'IN', 'IR', 'IS', 'IT', 'JM',
       'JP', 'KE', 'KR', 'KS', 'LB', 'LK', 'LT', 'LU', 'LV', 'MD', 'MM',
       'MT', 'MU', 'MX', 'MY', 'NG', 'NI', 'NL', 'NO', 'NZ', 'PE', 'PH',
       'PL', 'PT', 'QM', 'QZ', 'RE', 'RO', 'RS', 'RU', 'SE', 'SG', 'SI',
       'SK', 'SM', 'SW', 'TC', 'TH', 'TR', 'TW', 'UA', 'UK', 'UR', 'US',
       'UY', 'VE', 'VN', 'ZA', 'ZZ'], dtype=object)

Gruppierung der Songs nach den Ländercodes.

In [9]:
countries = {
    'AE': 'United Arab Emirates',
    'AR': 'Argentina',
    'AT': 'Austria',
    'AU': 'Australia',
    'AZ': 'Azerbaijan',
    'BA': 'Bosnia and Herzegovina',
    'BC': 'Canada (British Columbia)',
    'BE': 'Belgium',
    'BG': 'Bulgaria',
    'BR': 'Brazil',
    'BX': 'Benelux',
    'CA': 'Canada',
    'CH': 'Switzerland',
    'CI': 'Ivory Coast (Côte d\'Ivoire)',
    'CL': 'Chile',
    'CN': 'China',
    'CO': 'Colombia',
    'CS': 'Serbia and Montenegro',
    'CZ': 'Czech Republic',
    'DE': 'Germany',
    'DG': 'Germany (East)',
    'DK': 'Denmark',
    'EE': 'Estonia',
    'EG': 'Egypt',
    'ES': 'Spain',
    'FI': 'Finland',
    'FR': 'France',
    'FX': 'France (Metropolitan)',
    'GB': 'United Kingdom',
    'GN': 'Guinea',
    'GR': 'Greece',
    'GT': 'Guatemala',
    'GX': 'Greece (Ancient)',
    'HK': 'Hong Kong',
    'HR': 'Croatia',
    'HU': 'Hungary',
    'ID': 'Indonesia',
    'IE': 'Ireland',
    'IL': 'Israel',
    'IN': 'India',
    'IR': 'Iran',
    'IS': 'Iceland',
    'IT': 'Italy',
    'JM': 'Jamaica',
    'JP': 'Japan',
    'KE': 'Kenya',
    'KR': 'South Korea',
    'KS': 'Kosovo',
    'LB': 'Lebanon',
    'LK': 'Sri Lanka',
    'LT': 'Lithuania',
    'LU': 'Luxembourg',
    'LV': 'Latvia',
    'MD': 'Moldova',
    'MM': 'Myanmar (Burma)',
    'MT': 'Malta',
    'MU': 'Mauritius',
    'MX': 'Mexico',
    'MY': 'Malaysia',
    'NG': 'Nigeria',
    'NI': 'Nicaragua',
    'NL': 'Netherlands',
    'NO': 'Norway',
    'NZ': 'New Zealand',
    'PE': 'Peru',
    'PH': 'Philippines',
    'PL': 'Poland',
    'PT': 'Portugal',
    'QM': 'Marshall Islands', #second country code for the United States
    'QZ': 'Unknown or Invalid Area',
    'RE': 'Réunion',
    'RO': 'Romania',
    'RS': 'Serbia',
    'RU': 'Russia',
    'SE': 'Sweden',
    'SG': 'Singapore',
    'SI': 'Slovenia',
    'SK': 'Slovakia',
    'SM': 'San Marino',
    'SW': 'Sweden',
    'TC': 'Turks and Caicos Islands',
    'TH': 'Thailand',
    'TR': 'Turkey',
    'TW': 'Taiwan',
    'UA': 'Ukraine',
    'UK': 'United Kingdom',
    'UR': 'Uruguay',
    'US': 'United States',
    'UY': 'Uruguay',
    'VE': 'Venezuela',
    'VN': 'Vietnam',
    'ZA': 'South Africa',
    'ZZ': 'Unknown or Invalid Area'
}

Erstellen eines neuen Features `country`, welches den Namen des jeweiligen Landes beinhaltet.

In [10]:
by_isrc["country"] = by_isrc["country_code"].apply(lambda x: countries[x])

Gruppieren der Daten nach `country` und `country_code`. Dies soll genutzt werden, um zu analysieren, aus welchem Land am meisten Songs veröffentlicht worden sind.

In [11]:
grouped_countries = by_isrc.copy()
grouped_countries = grouped_countries[["country_code", "country"]]
grouped_countries["num_countries"] = 0
grouped_countries = grouped_countries.groupby(["country_code", "country"]).count()

In [12]:
grouped_countries = grouped_countries.sort_values("num_countries", ascending=False)

In [13]:
grouped_countries.reset_index(inplace=True)

In [14]:
# country und country_code werden kombiniert für eine bessere Darstellung
grouped_countries["country_comb"] = grouped_countries.apply(lambda x: f'{x["country"]}-{x["country_code"]}', axis=1)

In [15]:
fig = px.bar(grouped_countries[:30], x="country_comb", y="num_countries", template="plotly_dark")
fig.update_layout(
    height=500,
    title="Veröffentlichte Songs pro Land (Top 30)",
    xaxis_title="Land",
    yaxis_title="Veröffentlichte Songs"
)
fig.show()

Es sollen alle Lieder entfernt werden welche nicht aus englischsprachigen Länger (US, UK, CA, AU, Neuseeland) sowie Deutschland, Schweiz, Österreich, Italien und Schweden stammen.

In [16]:
important_country_codes = ["US", "GB", "CA", "AU", "NZ", "DE", "CH", "AT", "IT", "SE"]

# kleinere Inselgegenden von Amerika: Baker Island, Howland Island, Jarvis Island, Johnston Atoll, Kingman Reef, Midway Islands, Navassa Island, Palmyra Atoll, Wake Island
minor_insular_areas_us = ["XB", "XH", "XQ", "XU", "XM", "QM", "XV", "XL", "QW"]
important_country_codes.extend(minor_insular_areas_us)

`QM` wird heute als zusätzlicher Country code für Amerika verwendet, da der Code `US` sein Limit erreicht hat. Da dieser Beschluss jedoch aus 2010 stammt, sollten die Lieder der 80er nicht beeinflusst werden?

In [17]:
by_isrc[by_isrc["country_code"] == "QM"].sort_values("popularity", ascending=False)[["isrc", "genres", "name", "artists", "popularity", "chart_power"]]

,isrc,genres,name,artists,popularity,chart_power
274154,QMKHM1600219,"['hard-rock', 'metal', 'rock']",master of puppets,metallica,80,NaN
274107,QMKHM1600096,"['hard-rock', 'metal', 'rock']",for whom the bell tolls - remastered,metallica,75,NaN
271243,QMFME1326440,"['latin', 'pop']",tu dama de hierro,marisela,71,NaN
274108,QMKHM1600097,"['hard-rock', 'metal', 'rock']",fade to black - remastered,metallica,70,NaN
274057,QMKHM1600034,"['hard-rock', 'metal', 'rock']",seek & destroy - remastered,metallica,69,NaN
...,...,...,...,...,...,...
267541,QM7281628419,['tango'],fuegos artificiales,juan d'arienzo y su orquesta típica,0,NaN
267551,QM7281713701,['jazz'],raincheck,nick brignola,0,NaN
267552,QM7281713702,['jazz'],tenderly,nick brignola,0,NaN
267553,QM7281713703,['jazz'],hurricane connie,nick brignola,0,NaN


Herausfiltern der Lieder aus den "wichtigen" Länder.

In [18]:
reduced_isrc = by_isrc.copy()
reduced_isrc = reduced_isrc[reduced_isrc["country_code"].isin(important_country_codes)]
reduced_isrc["genres"] = reduced_isrc["genres"].apply(literal_eval)

In [19]:
print(f"Die Anzahl der Lieder hat sich von {by_isrc.shape[0]} auf {reduced_isrc.shape[0]} verringert.")

Die Anzahl der Lieder hat sich von 416154 auf 251101 verringert.


Gruppieren der Daten nach den Genres => Welche Genres waren in den wichtigen Ländern relevant?

In [20]:
def filter_important_genres(df):
  important_genres = df.copy()
  important_genres = important_genres.explode("genres")
  important_genres["num"] = 0
  important_genres = important_genres[["genres", "num"]]
  important_genres = important_genres.groupby("genres").count()
  important_genres.reset_index(inplace=True)
  important_genres = important_genres.sort_values("num", ascending=False)
  return important_genres

In [21]:
filter_important_genres(reduced_isrc)

,genres,num
16,classical,63432
87,rock,60598
77,pop,48841
38,german,28985
62,jazz,22632
...,...,...
28,dubstep,2
12,cantopop,1
53,idm,1
69,metalcore,1


Am meisten Lieder wurden für das Genre Klassik veröffentlicht. Da dieses Genre nicht wirklich relevant ist, müssen zunächst die irrelevanten Genres entfernt werden. Hiebei gibt es zwei verschiedene Wege. Verfügt ein Künstler sowohl über ein erwünschtes als auch ein unerwünschtes Genre, so können entweder alle Songs von ihm behalten oder entfernt werden. Zunächst wird geprüft, wie häufig ein solcher Fall eintritt.

In [22]:
unwanted_genres = ["classical", "jazz", "folk", "french", "turkish", "gospel", "samba", "piano", "mpb", "sertanejo", "pagode", "sleep", "forro", "malay", "anime",
                 "j-idol", "comedy", "mandopop", "cantopop", "show-tunes", "emo", "romance", "j-dance", "chill", "world-music", "iranian", "idm", "metalcore",
                 "hardstyle", "opera", "k-pop", "j-pop", "j-rock", "happy"]

In [23]:
occ_unwanted_and_wanted = by_isrc.copy()
occ_unwanted_and_wanted = occ_unwanted_and_wanted[["genres"]]
occ_unwanted_and_wanted["genres"] = occ_unwanted_and_wanted["genres"].apply(literal_eval)
occ_unwanted_and_wanted = occ_unwanted_and_wanted.explode("genres")

In [24]:
wanted_genres = list(set(occ_unwanted_and_wanted["genres"].unique()).difference(unwanted_genres))

In [25]:
def filter_row(row):
    has_wanted_genre = any(genre in row['genres'] for genre in wanted_genres)
    has_not_wanted_genre = any(genre in row['genres'] for genre in unwanted_genres)
    return has_wanted_genre and has_not_wanted_genre

In [26]:
occ_unwanted_and_wanted_df = reduced_isrc[reduced_isrc.apply(filter_row, axis=1)]

In [27]:
print(f"Insgesamt gibt es {occ_unwanted_and_wanted_df.shape[0]} Lieder, welche über ein erwünschtes und ein unerwünschtes Genre verfügen.")

Insgesamt gibt es 48318 Lieder, welche über ein erwünschtes und ein unerwünschtes Genre verfügen.


In [28]:
print(f"Hierbei gibt es {len(list(occ_unwanted_and_wanted_df.genres.astype(str).unique()))} verschiedene Kombinationen der Genres.")

Hierbei gibt es 582 verschiedene Kombinationen der Genres.


# Visualisierung Kombination

Es werden alle Songs entfernt, welche mindestens ein unerwünschtes Feature beinhalten.

In [29]:
by_isrc_sub = by_isrc.copy()
by_isrc_sub = by_isrc_sub[['isrc', 'genres']]
by_isrc_sub = by_isrc_sub[~by_isrc_sub['genres'].str.contains('|'.join(unwanted_genres))]
by_isrc_sub['genres'] = by_isrc_sub['genres'].apply(literal_eval)

Das neu entstandene DataFrame soll nun mit dem bereits auf die wichtigen Länder reduzierten DataFrame gemerged werden.

In [30]:
#reduced_isrc.drop(columns="genres", inplace=True)

In [31]:
reduced_isrc.shape, by_isrc_sub.shape

((251101, 27), (220702, 2))

In [32]:
reduced_isrc = reduced_isrc.merge(by_isrc_sub[["isrc"]], how="inner", on="isrc")

In [33]:
print(f"Die Anzahl der Songs hat sich weiter auf {reduced_isrc.shape[0]} verringert. Die Anzahl der Features ist bei {reduced_isrc.shape[1]} geblieben.")

Die Anzahl der Songs hat sich weiter auf 139629 verringert. Die Anzahl der Features ist bei 27 geblieben.


In [34]:
important_genres = filter_important_genres(reduced_isrc)
important_genres = important_genres.sort_values("num", ascending=False)
important_genres.head(10)

,genres,num
61,rock,52805
51,pop,43525
56,punk,16552
14,country,12353
47,metal,11743
35,hard-rock,9228
29,german,8980
20,disco,8065
70,swedish,7964
68,soul,7289


In [35]:
fig = px.bar(important_genres[:30], x="genres", y="num", template="plotly_dark")
fig.update_layout(
    height=500,
    title="Veröffentlichte Songs pro Genre in den wichtigen Ländern (Top 30)",
    xaxis_title="Genre",
    yaxis_title="Veröffentlichte Songs"
)
fig.show()

In [36]:
reduced_isrc.to_csv(f"{PATH}/reduced_isrc.csv")

## Auffindbarkeiten prüfen (in db und auf Spotify)

In [37]:
!pip install colab-env --upgrade

In [38]:
!pip install spotipy --upgrade

In [65]:
import colab_env
import os
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import requests
import json
from numpy import NaN
from tqdm import tqdm

In [40]:
chart_power_df = pd.read_excel(f"{PATH}/chart-power-scores_80s.xlsx")

In [41]:
SPOTIFY_CLIENT_ID = os.getenv("SPOTIFY_CLIENT_ID")
SPOTIFY_CLIENT_SECRET = os.getenv("SPOTIFY_CLIENT_SECRET")

In [42]:
client_credentials_manager = SpotifyClientCredentials(client_id=SPOTIFY_CLIENT_ID, client_secret=SPOTIFY_CLIENT_SECRET)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

Abfrage aller Lieder für Boney M.

In [56]:
def query(year: int, offset: int):
  res = sp.search(q=f"artist:Boney M. year:{year}", type="track", limit=10, offset=offset, market="DE")
  if (len(res["tracks"]["items"]) == 10):
    return query(year, offset + 10)
  return len(res["tracks"]["items"]) + offset

In [51]:
boney_songs = pd.DataFrame()
for year in range(1980, 1990):
  boney_songs = pd.concat([boney_songs, pd.DataFrame([[year, query(year, 0)]], columns=["year", "songs"])])

In [45]:
fig = px.bar(boney_songs, x="year", y="songs", title="Songs veröffentlich von Boney M. in den 80er Jahren")
fig.update_layout(
    height=500,
    template='plotly_dark',
    xaxis_title="Jahr",
    yaxis_title="Veröffentlichte Songs"
)
fig.show()

Abfrage aller Lieder von Künstlern, welche in den 80ern in den Charts waren

In [58]:
chart_power_df["Artist"].unique()

array(['Lionel Richie', 'Harold Faltermeyer', 'Kaoma', 'A-ha',
       'Whitney Houston', 'Louis Armstrong', 'Michael Jackson',
       'Bon Jovi', 'Rick Astley', 'Cyndi Lauper', 'Europe', 'Scorpions',
       'The Police', 'Bonnie Tyler', 'George Michael', 'Survivor',
       'Boy George', 'Glenn Medeiros', 'Hot Chocolate', "T'Pau", 'Black',
       'Joe Cocker', 'Caroline Loeb', 'Toto', 'Eurythmics', 'Roxette',
       'Culture Club', 'Pink Floyd', 'Modern Talking', 'Queen',
       'Bryan Adams', 'USA for Africa', 'Ben E. King', 'AC/DC',
       'Paul McCartney', 'Madonna', 'Martika',
       'Bill Medley & Jennifer Warnes', 'Phil Collins', 'Elton John',
       'Wham!', 'Alphaville', 'Richard Marx', 'Berlin',
       'Bob Marley & The Wailers', 'Olivia Newton-John',
       'Queen & David Bowie', 'Blondie', 'Real Life', 'Men At Work',
       'Depeche Mode', 'Kiss', 'Village People', 'Heart',
       'Bruce Springsteen', 'ABBA', 'Genesis', 'Ray Parker Jr.',
       'Alice Cooper', 'Kool & The Gan

Wenn mehrere Künstler zusammen einen Song veröffentlich haben, sind diese hier mit einem `&` getrennt. Dies muss zunächst aufgelöst werden.

In [59]:
chart_power_df["Artist"] = chart_power_df["Artist"].str.split(" & ")
chart_power_df_explode = chart_power_df.explode("Artist", ignore_index=True)

In [60]:
chart_power_df_explode

,Year,Edates,Song,Artist,Points
0,1984,1984-04-16,Hello,Lionel Richie,1330
1,1985,1985-05-27,Axel F.,Harold Faltermeyer,1467
2,1989,1989-09-04,Lambada,Kaoma,1662
3,1986,1986-01-06,Take On Me,A-ha,701
4,1985,1985-10-14,Take On Me,A-ha,1022
...,...,...,...,...,...
2138,1987,1987-07-27,Mit Dir Des Wär Mei Leben,Pierre Cosso,702
2139,1987,1987-03-23,Ever Fallen In Love,Johnny Hates Jazz,830
2140,1987,1987-06-08,If You Let Me Stay,Pepsi,773
2141,1987,1987-06-08,If You Let Me Stay,Shirlie,773


In [67]:
def filter_track_features(track, genre):
    '''
    Filters the relevant features of a track in returns them in JSON object.

    Parameter
    ---------
    track: Object
        Track returend by the spotify API

    genre: string
        Genre that should be used

    Return
    ------
    relevant_features: Object
        JSON Object that contains the relevant featues
    '''

    features = sp.audio_features(track['id'])[0]

    external_ids = track['external_ids'] if 'external_ids' in track else {}
    isrc = external_ids['isrc'] if 'isrc' in external_ids else NaN
    artist_names = []

    if 'artists' in track and type(track['artists']) == list:
        for artist in track['artists']:
            if 'name' in artist:
                artist_names.append(artist['name'])

    artist_names = ','.join(artist_names)

    if 'album' in track:
        album = track['album']['name'] if 'name' in track['album'] else NaN
        release_date = track['album']['release_date'] if 'release_date' in track['album'] else NaN
        release_date_precision = track['album']['release_date_precision'] if 'release_date_precision' in track['album'] else NaN
    else:
         album = NaN
         release_date = NaN
         release_date_precision = NaN


    track_name = track['name'] if 'name' in track else NaN
    if track_name != NaN:
        points = chart_power_df.loc[(chart_power_df.Song == track_name.lower()) & (chart_power_df.Artist == artist_names.lower())]['Points']
        if points.empty:
             points = NaN
        else:
            points = int(points)
    else:
         points = NaN


    return {
        'name': track_name,
        'artists': artist_names,
        'album': album,
        'release_date': release_date,
        'release_date_precision': release_date_precision,
        'spotify_id': track['id'] if 'id' in track else NaN,
        'chart_power': points,
        'uri': track['uri'] if 'uri' in track else NaN,
        'popularity': track['popularity'] if 'popularity' in track else NaN,
        'genres': genre,
        'danceability': features['danceability'] if 'danceability' in features else NaN,
        'energy': features['energy'] if 'energy' in features else NaN,
        'key': features['key'] if 'key' in features else NaN,
        'loudness': features['loudness'] if 'loudness' in features else NaN,
        'mode': features['mode'] if 'mode' in features else NaN,
        'speechiness': features['speechiness'] if 'speechiness' in features else NaN,
        'acousticness': features['acousticness'] if 'acousticness' in features else NaN,
        'instrumentalness': features['instrumentalness'] if 'instrumentalness' in features else NaN,
        'liveness': features['liveness'] if 'liveness' in features else NaN,
        'valence': features['valence'] if 'valence' in features else NaN,
        'tempo': features['tempo'] if 'tempo' in features else NaN,
        'duration_ms': features['duration_ms'] if 'duration_ms' in features else NaN,
        'time_signature': features['time_signature'] if 'time_signature' in features else NaN,
        'isrc': isrc,
    }

In [68]:
def get_number_of_tracks(release_year, artist, genres):
    '''
    Retrieves the number of tracks the spotfiy API returns for a specific query.
    '''
    try:
        result = sp.search(q=f'year:{release_year} artist:{artist} genre:{genre}', type='track', limit=1, offset=0, market='DE')
        tracks = result['tracks'] if 'tracks' in result else ''
        return tracks['total'] if 'total' in tracks else 0
    except Exception as e:
        print(e)
        #error_logger.error(e)
    return 0

In [69]:
genres = ['chicago-house', 'hard-rock', 'rock', 'power-pop', 'guitar', 'hardcore',
          'groove', 'songwriter', 'garage', 'disco', 'grunge', 'ambient',
          'minimal-techno', 'brazil', 'indie', 'detroit-techno', 'electronic',
          'edm', 'dubstep', 'children', 'drum-and-bass', 'indian', 'death-metal',
          'club', 'salsa', 'bluegrass', 'metal', 'reggae', 'tango', 'black-metal',
          'british', 'german', 'punk', 'reggaeton', 'alternative', 'soul', 'r-n-b',
          'goth', 'pop-film', 'breakbeat', 'indie-pop', 'heavy-metal', 'dance',
          'honky-tonk', 'dancehall', 'dub', 'singer-songwriter', 'spanish',
          'deep-house', 'rock-n-roll', 'techno', 'hip-hop', 'punk-rock', 'industrial',
          'afrobeat', 'trip-hop', 'funk', 'blues', 'swedish', 'latino', 'alt-rock',
          'country', 'acoustic', 'trance', 'grindcore', 'ska', 'house', 'progressive-house',
          'new-age', 'electro', 'rockabilly', 'party', 'pop', 'synth-pop', 'latin',
          'psych-rock']

In [71]:
artist_over_1000 = []
df = pd.DataFrame()
with tqdm(total=944*len(genres)) as pbar:
  for artist in list(chart_power_df_explode["Artist"].unique()):
    for genre in genres:
      total_results = get_number_of_tracks("1980-1989", artist, genre)
      if total_results < 1000:
        offset = 0
        while offset < total_results:
          try:
            track_features = []
            result = sp.search(q=f"year:1980-1989 artist:{artist} genre:{genre}", type="track", limit=50, offset=offset, market="DE")
            tracks = result["tracks"] if "tracks" in result else ""
            if "items" in tracks:
              for track in tracks["items"]:
                features = filter_track_features(track, genre)
                track_features.append(features)
              offset += 50
              df = pd.concat([df, pd.DataFrame(track_features)], ignore_index=True)
            else:
              continue
          except Exception as e:
            print(e)
      else:
        print(f"Artist {artist} has published over 1000 songs in the 80s")
        artist_over_1000.append(artist)

      pbar.update(1)

  0%|          | 2/71744 [00:00<1:05:51, 18.16it/s]ERROR:spotipy.client:Max Retries reached


(429, -1, '/v1/audio-features/?ids=0snPJPxkk0MbTc0xeUvAPt:\n Max Retries')


ERROR:spotipy.client:Max Retries reached


(429, -1, '/v1/audio-features/?ids=0snPJPxkk0MbTc0xeUvAPt:\n Max Retries')


ERROR:spotipy.client:Max Retries reached


(429, -1, '/v1/audio-features/?ids=0snPJPxkk0MbTc0xeUvAPt:\n Max Retries')


ERROR:spotipy.client:Max Retries reached


(429, -1, '/v1/audio-features/?ids=0snPJPxkk0MbTc0xeUvAPt:\n Max Retries')


  0%|          | 2/71744 [00:10<104:55:31,  5.27s/it]


KeyboardInterrupt: ignored